<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/GPT4Dev_ch03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 API 參數解析與錯誤處理

## 3-1 事前準備

安裝必要的套件與匯入相關模組後建立用戶端物件

In [ ]:
!pip install openai
!pip install rich
import openai
from google.colab import userdata
from rich import print as pprint
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

## 3-2 控制生成訊息與 token 數量

### 指定生成的訊息數量 - n

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[{"role": "user", "content": "你好"}],
  n=2
)

pprint(reply)

for choice in reply.choices:
    print(choice.index, choice.message.content)

### 設定詞彙黑名單 - stop

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "你好"}],
  stop=['好']
)

print(reply.choices[0].message.content)
print(reply.choices[0].finish_reason)

### 設定回覆語句的 tokens 數量上限 - max_tokens

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"user", "content": "您好!"}
    ],
    max_tokens = 5
)

print(reply.choices[0].message.content)
print(reply.choices[0].finish_reason)
print(reply.usage.completion_tokens)

In [ ]:
!pip install tiktoken
import tiktoken
encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')

In [ ]:
encoder.encode("您好！有什")

超過模型限制的 tokens 數

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo-1106",
    messages = [
        {"role":"user", "content": "你好"}
    ],
    max_tokens = 16380
)

## 3-3 控制回覆內容的變化性

### 讓回覆更具彈性 - temperature

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[{"role": "user", "content": "嗨！"}],
  temperature=0,
  n=2
)

for choice in reply.choices:
    print(choice.index, choice.message.content)

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[{"role": "user", "content": "嗨！"}],
  temperature=2,
  n=2
)

for choice in reply.choices:
    print(choice.index, choice.message.content)

### 控制詞彙的豐富度 - top_p

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "嗨！"}],
  top_p=0,
  n=2
)

for choice in reply.choices:
    print(choice.index, choice.message.content)

### 控制詞彙的重複性 - presence_penalty 與 frequency_penalty

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[{
    "role": "user",
    "content": "台北是什麼樣的城市？"
  }],
  temperature=1,
  presence_penalty=-2,
)

print(reply.choices[0].message.content)

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[{
    "role": "user",
    "content": "台北是什麼樣的城市？"
  }],
  temperature=1,
  presence_penalty=2,
)

print(reply.choices[0].message.content)

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[{
      "role": "user",
      "content": "台北是什麼樣的城市？"}],
  temperature=1,
  frequency_penalty=-2,
)

print(reply.choices[0].message.content)

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[{
      "role": "user",
      "content": "台北是什麼樣的城市？"}],
  temperature=1,
  frequency_penalty=2,
)

print(reply.choices[0].message.content)

### 調整特定 token 的分數 - logi-bias


In [ ]:
encoder.encode('你好')

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "你好"}],
  temperature=1,
  logit_bias={
      53901: -100,
      57668: -100
  },
)

print(reply.choices[0].message.content)

In [ ]:
encoder.encode('哈')

In [ ]:
reply = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "你好"}],
  temperature=1,
  logit_bias={
      99771: 100,
  },
)

print(reply.choices[0].message.content)

### 固定可預測的輸出 - seed

使用亂數種子固定輸出, 不保證, 自己要檢查回覆指紋。1106 的模型才提供。

In [ ]:
replies = client.chat.completions.create(
  model="gpt-3.5-turbo-0613",
  messages=[{
    "role": "user",
    "content": "請用兩句話介紹台北市"
  }],
  seed=1
)

print(replies.system_fingerprint)
print(replies.choices[0].message.content)

In [ ]:
replies = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[{
    "role": "user",
    "content": "請用兩句話介紹台北市"
  }],
  seed=2
)

print(replies.system_fingerprint)
print(replies.choices[0].message.content)

## 3-4 串流輸出

### 可循序傳回結果的迭代器 (iterator) - stream


In [ ]:
replies = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{
    "role": "user",
    "content": "你好"
  }],
  stream=True,
)

for reply in replies:
    pprint(reply)

In [ ]:
replies = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{
    "role": "user",
    "content": "台北是什麼樣的城市？"
  }],
  stream=True
)

for reply in replies:
    print(reply.choices[0].delta.content or '', end='')

### 串流多個語句

In [ ]:
replies = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{
    "role": "user",
    "content": "你好"
  }],
  stream=True,
  n=2
)

for reply in replies:
    print(reply.choices[0].delta.content or '', end='')

##3-5 進階控制

### 控制輸出格式 - response_format


In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo-1106",
    messages = [{
        "role":"user",
        "content": "台灣最高的山高度是多少？"
                   "請分別告訴我山名和高度"
        }
    ]
)

print(reply.choices[0].message.content)

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo-1106",
    messages = [
        {"role":"user",
         "content": "台灣最高的山高度是多少？"
         "請分別告訴我山名和高度"},
        {"role":"system", "content": "請用 json 格式回覆"}
    ],
    response_format={'type': 'json_object'} # or 'text'
)

print(reply.choices[0].message.content)

### 取得底層 HTTP 回應內容

In [ ]:
# 取得原始 HTTP 回覆內容
reply = client.chat.completions.with_raw_response.create(
    model = "gpt-3.5-turbo-1106",
    # model = "gpt-4",
    messages = [
        {"role":"user", "content": "你好"}
    ]
)

In [ ]:
import json
print(reply.status_code)
print('------')
print(reply.text) # JSON 格式文字
print('------')
reply_dic = json.loads(reply.text) # 轉成 Python 字典
pprint(reply_dic)

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    # model = "gpt-4",
    messages = [
        {"role":"user", "content": "你好"}
    ]
)

pprint(reply.model_dump())

### 有眼睛的模型 - gpt-4-vision (GPT-4V)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "圖片裡有什麼？"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://flagtech.github.io/F3762/"
                               "images/cat1.jpg",
                        'detail': 'high'
                    },
                },
            ],
        }
    ],
    max_tokens=1000,
)

pprint(response.choices[0].message.content)

In [ ]:
import requests
r = requests.get(
    'https://flagtech.github.io/F3762/images/cat2.jpg'
)

with open('cat2.jpg', 'wb') as f:
    f.write(r.content)

In [ ]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
img = encode_image('cat2.jpg')

response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "圖片裡有什麼？"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{img}",
                        'detail': 'high'
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0].message.content)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "這些圖片裡相同的是什麼？"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://flagtech.github.io/F3762/"
                               "images/cat1.jpg",
                        'detail': 'high'
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://flagtech.github.io/F3762/"
                               "images/cat2.jpg",
                        'detail': 'high'
                    },
                },
            ],
        }
    ],
    max_tokens=1000,
)

pprint(response.choices[0].message.content)

## 3-5 錯誤處理與使用限制

In [ ]:
'''
Exception
+--OpenAIError
   +--APIError ◆ message: str
      |      ◆ request: httpx.Request
      +--APIResponseValidationError ◆ response: httpx.Response
      |  (回覆內容格式有誤)            ◆ status_code: int
      +--APIStatusError ◆ response: httpx.Response
      |  |            ◆ status_code: int
      |  +--BadRequestError (請求參數或是格式錯誤)
      |  +--AuthenticationError (金鑰認證有問題)
      |  +--PermissionDeniedError
      |  +--NotFoundError
      |  +--ConflictError
      |  +--UnprocessableEntityError
      |  +--RateLimitError (超過次數限制)
      |  +--InternalServerError
      +--APIConnectionError (無法連線)
         +--APITimeoutError (連線逾時)
'''

### 使用例外機制處理錯誤

In [ ]:
try:
    reply = client.chat.completions.create(
        model = "gpt-3.5-turbo-1106",
        messages = [
            {"role":"user", "content": "你好"}
        ],
        max_tokens = 20000
    )
    print(reply.choices[0].message.content)

except openai.APIError as err:
    print(err.message)

In [ ]:
try:
    reply = client.chat.completions.create(
        model = "gpt-3.5-turbo-1106",
        messages = [
            {"role":"user", "content": "你好"}
        ],
        max_tokens = 20000
    )
    print(reply.choices[0].message.content)

except openai.APIStatusError as err:
    err_json = err.response.json()
    print(f"錯誤類型：{err_json['error']['type']}")
    print(f"錯誤碼：{err_json['error']['code']}")
    print(f"錯誤訊息：{err_json['error']['message']}")